In [1]:
# Import dependencies
import pandas as pd
import numpy as np
import re
import time

In [8]:
# Create DataFrame from CSV file
nlp_df = pd.read_csv('../Data/nlp_df.csv')
nlp_df = nlp_df.drop(['non_alpha_words'], axis=1)
nlp_df.head(3)

song                 song_id  \
0                                 willow  0lx2cLdOt3piJbcaXIV74f   
1  Stay Next To Me (with Chelsea Cutler)  6SGG5AxHShqSYiV9fCWpZz   
2                            WITHOUT YOU  27OeeYzk6klgBh83TSvGMA   

          artist               artist_id category  category_id  popularity  \
0   Taylor Swift  06HL4z0CvFAxyc27GXpf02      pop            8          93   
1     Quinn XCII  3ApUX1o6oSz321MMECyIYd      pop            8          78   
2  The Kid LAROI  2tIP7SsRs7vjIcLrU85W8J      pop            8          95   

                           genres  audio_ft_danceability  audio_ft_energy  \
0                ['dance', 'pop']                  0.392            0.574   
1  ['indie', 'pop', 'electropop']                  0.581            0.584   
2                  ['australian']                  0.662            0.413   

   ...  audio_ft_acousticness  audio_ft_instrumentalness  audio_ft_liveness  \
0  ...                 0.8330                    0.00179              0.145   
1  ...                 0.0805                    0.00000              0.366   
2  ...                 0.2130                    0.00000              0.134   

   audio_ft_valence  audio_ft_tempo  audio_ft_duration_ms  \
0             0.529          81.112              214707.0   
1             0.756         179.954              206046.0   
2             0.467          93.005              161385.0   

   audio_ft_time_signature                                             lyrics  \
0                      4.0   Im like the water when your ship rolled in th...   
1                      4.0   Didnt even wanna go out whyd you call me ? Iv...   
2                      4.0   You cut out a piece of me and now I bleed int...   

                                            filtered   language  
0  ['', 'im', 'like', 'water', 'ship', 'rolled', ...  (1, 'en')  
1  ['', 'didnt', 'even', 'wanna', 'go', 'whyd', '...  (1, 'en')  
2  ['', 'cut', 'piece', 'bleed', 'internally', 'l...  (1, 'en')  

[3 rows x 24 columns]

In [9]:
# Create a list of all words, word counts, unique word counts, and filtered words
t0 = time.time()
words_list = []
word_counts = []
unique_word_counts = []
filtered_words_list = []
for index, row in nlp_df.iterrows():
    filtered_words = row['filtered']
    filtered_words = filtered_words.replace(',', '').replace("'", '')
    filtered_words = filtered_words.replace('[', '').replace(']', '')
    filtered_words = filtered_words.replace('#', '').replace('&nbsp', '')
    filtered_words = filtered_words.replace('?', '? ').replace(',', '')
    filtered_words = filtered_words.replace('/', ' ')
    filtered_words = filtered_words.replace('\\u200a', '').replace('\\u200b', '')
    filtered_words = filtered_words.replace('\\u2063', '').replace('\u202f', '')
    filtered_words = filtered_words.replace('\\u2028', ' ').replace('\\u2008', ' ')
    while ('\\u200e' in filtered_words) or ('\\xa0' in filtered_words):
        filtered_words = filtered_words.replace('\\u200e', '')
        filtered_words = filtered_words.replace('\\xa0', '')
    filtered_words_list.append(filtered_words)
    words = filtered_words.strip().split(' ')
    word_counts.append(len(words))
    unique_words = list(set(words))
    unique_word_counts.append(len(unique_words))
    words_list.extend(unique_words)
word_columns = list(set(words_list))
t1 = time.time()
print(f'Run time: {t1-t0} seconds')
len(word_columns)

Run time: 1.5752971172332764 seconds


36234

In [10]:
# Add word_count and unique_word_count columns and replace the filtered column
nlp_df['word_count'] = word_counts
nlp_df['unique_word_count'] = unique_word_counts
nlp_df['filtered'] = filtered_words_list
nlp_df.head(1)

song                 song_id        artist               artist_id  \
0  willow  0lx2cLdOt3piJbcaXIV74f  Taylor Swift  06HL4z0CvFAxyc27GXpf02   

  category  category_id  popularity            genres  audio_ft_danceability  \
0      pop            8          93  ['dance', 'pop']                  0.392   

   audio_ft_energy  ...  audio_ft_liveness  audio_ft_valence  audio_ft_tempo  \
0            0.574  ...              0.145             0.529          81.112   

   audio_ft_duration_ms  audio_ft_time_signature  \
0              214707.0                      4.0   

                                              lyrics  \
0   Im like the water when your ship rolled in th...   

                                            filtered   language  word_count  \
0   im like water ship rolled night rough surface...  (1, 'en')         227   

   unique_word_count  
0                 87  

[1 rows x 26 columns]

In [11]:
# Remove songs with less than 25 unique words
nlp_df = nlp_df[nlp_df['unique_word_count']>=25]
len(nlp_df)

8041

# Examine word counts by genre

In [12]:
# Remove the language column
nlp_df = nlp_df.drop('language', axis=1)

In [23]:
# Drop songs with incorrect lyrics from the DataFrame
nlp_df = nlp_df[(nlp_df['unique_word_count']!=3878) & (nlp_df['unique_word_count']!=1153)]
nlp_df = nlp_df[(nlp_df['unique_word_count']!=1000) & (nlp_df['unique_word_count']!=880)]
nlp_df = nlp_df[(nlp_df['unique_word_count']!=842) & (nlp_df['unique_word_count']!=607)]
nlp_df = nlp_df[(nlp_df['unique_word_count']!=569) & (nlp_df['word_count']!=1383)]
len(nlp_df)

8031

In [24]:
# Describe the distribution of unique word counts for each cateogry
word_distributions = {}
blues_unique_words = nlp_df[nlp_df['category']=='blues']['word_count'].describe()
word_distributions['blues'] = blues_unique_words

classical_unique_words = nlp_df[nlp_df['category']=='classical']['word_count'].describe()
word_distributions['classical'] = classical_unique_words

country_unique_words = nlp_df[nlp_df['category']=='country']['word_count'].describe()
word_distributions['country'] = country_unique_words

funk_unique_words = nlp_df[nlp_df['category']=='funk']['word_count'].describe()
word_distributions['funk'] = funk_unique_words

hiphop_unique_words = nlp_df[nlp_df['category']=='hiphop']['word_count'].describe()
word_distributions['hiphop'] = hiphop_unique_words

indie_alt_unique_words = nlp_df[nlp_df['category']=='indie_alt']['word_count'].describe()
word_distributions['indie_alt'] = indie_alt_unique_words

jazz_unique_words = nlp_df[nlp_df['category']=='jazz']['word_count'].describe()
word_distributions['jazz'] = jazz_unique_words

metal_unique_words = nlp_df[nlp_df['category']=='metal']['word_count'].describe()
word_distributions['metal'] = metal_unique_words

pop_unique_words = nlp_df[nlp_df['category']=='pop']['word_count'].describe()
word_distributions['pop'] = pop_unique_words

punk_unique_words = nlp_df[nlp_df['category']=='punk']['word_count'].describe()
word_distributions['punk'] = punk_unique_words

rnb_unique_words = nlp_df[nlp_df['category']=='rnb']['word_count'].describe()
word_distributions['rnb'] = rnb_unique_words

rock_unique_words = nlp_df[nlp_df['category']=='rock']['word_count'].describe()
word_distributions['rock'] = rock_unique_words

romance_unique_words = nlp_df[nlp_df['category']=='romance']['word_count'].describe()
word_distributions['romance'] = romance_unique_words

soul_unique_words = nlp_df[nlp_df['category']=='soul']['word_count'].describe()
word_distributions['soul'] = soul_unique_words
len(word_distributions)

14

In [25]:
# Print the distributions for each genre
word_distributions

{'blues': count    424.000000
 mean     119.740566
 std       47.975256
 min       36.000000
 25%       84.000000
 50%      111.500000
 75%      147.000000
 max      364.000000
 Name: word_count, dtype: float64,
 'classical': count     93.000000
 mean     150.860215
 std      116.015328
 min       28.000000
 25%       81.000000
 50%      123.000000
 75%      183.000000
 max      823.000000
 Name: word_count, dtype: float64,
 'country': count    1257.000000
 mean      166.240255
 std        53.697886
 min        30.000000
 25%       131.000000
 50%       166.000000
 75%       197.000000
 max       452.000000
 Name: word_count, dtype: float64,
 'funk': count    282.000000
 mean     202.886525
 std      100.706656
 min       40.000000
 25%      128.250000
 50%      188.500000
 75%      251.500000
 max      583.000000
 Name: word_count, dtype: float64,
 'hiphop': count    680.000000
 mean     352.647059
 std      124.737895
 min       59.000000
 25%      265.000000
 50%      343.000000
 75

In [26]:
# Create a dictionary of mean word distributions
categories = list(word_distributions.keys())
means = []
for category in word_distributions:
    means.append(word_distributions[category]['mean'])
mean_unique_word_counts = dict(zip(categories, means))
mean_unique_word_counts = dict(sorted(mean_unique_word_counts.items(), key=lambda item: item[1], reverse=True))
mean_unique_word_counts

{'hiphop': 352.6470588235294,
 'rnb': 232.48342541436463,
 'funk': 202.88652482269504,
 'pop': 192.4992784992785,
 'soul': 180.97810218978103,
 'country': 166.24025457438344,
 'rock': 157.45795918367347,
 'punk': 154.72934973637962,
 'classical': 150.86021505376345,
 'metal': 146.5,
 'romance': 144.70250896057348,
 'indie_alt': 141.33939393939394,
 'jazz': 123.6010101010101,
 'blues': 119.74056603773585}

In [27]:
# Find songs with the highest and lowest word counts
print('Song with the most words:')
display(nlp_df[nlp_df['word_count']==max(nlp_df['word_count'])][['song', 'artist', 'category', 'lyrics', 'word_count']])

print('Song with the least words:')
display(nlp_df[nlp_df['word_count']==min(nlp_df['word_count'])][['song', 'artist', 'category', 'lyrics', 'word_count']])


Song with the most words:


song  artist category  \
1351  Rap God  Eminem   hiphop   

                                                 lyrics  word_count  
1351   Look I was gonna go easy on you not to hurt y...         909

Song with the least words:


song     artist category  \
2547  Golf Hill Drive  Boys Life     rock   

                                                 lyrics  word_count  
2547  Stuck in this life thats not my own And whoeve...          26

In [28]:
# Create DataFrame of word count distributions
ranges = []
for category in word_distributions:
    ranges.append(word_distributions[category]['max'] - word_distributions[category]['min'])

columns = ['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max', 'range']
df_rows = []
for index, category in enumerate(word_distributions):
    distribution = list(word_distributions[category])
    distribution.extend([ranges[index]])
    df_rows.append(distribution)
word_count_df = pd.DataFrame(df_rows, index=list(word_distributions.keys()), columns=columns)
word_count_df

count        mean         std   min     25%    50%     75%    max  \
blues       424.0  119.740566   47.975256  36.0   84.00  111.5  147.00  364.0   
classical    93.0  150.860215  116.015328  28.0   81.00  123.0  183.00  823.0   
country    1257.0  166.240255   53.697886  30.0  131.00  166.0  197.00  452.0   
funk        282.0  202.886525  100.706656  40.0  128.25  188.5  251.50  583.0   
hiphop      680.0  352.647059  124.737895  59.0  265.00  343.0  431.25  909.0   
indie_alt   660.0  141.339394   59.928308  34.0   99.00  132.0  170.00  469.0   
jazz        198.0  123.601010   79.526948  32.0   76.00  107.0  144.00  566.0   
metal       898.0  146.500000   62.602518  30.0  103.00  136.0  176.75  485.0   
pop         693.0  192.499278   80.097008  36.0  145.00  181.0  224.00  815.0   
punk        569.0  154.729350   59.161844  36.0  114.00  148.0  190.00  562.0   
rnb         362.0  232.483425   93.733435  60.0  167.25  215.5  287.75  658.0   
rock       1225.0  157.457959   61.944011  26.0  117.00  149.0  190.00  528.0   
romance     279.0  144.702509   59.670929  38.0  104.00  138.0  172.00  428.0   
soul        411.0  180.978102   82.099951  44.0  123.00  166.0  226.00  775.0   

           range  
blues      328.0  
classical  795.0  
country    422.0  
funk       543.0  
hiphop     850.0  
indie_alt  435.0  
jazz       534.0  
metal      455.0  
pop        779.0  
punk       526.0  
rnb        598.0  
rock       502.0  
romance    390.0  
soul       731.0

In [29]:
# Save DataFrame to CSV
word_count_df.to_csv('../Data/Analysis/word_count.csv')

# Conclusions about word distributions by genre
- The hiphop genre has the highest mean word count of 353 words
- The blues genre has the lowest mean word count of 120 words
- Genres with the highest average word counts:
    - Hiphop: 353 words
    - R&B: 232 words
    - Funk: 203 words
- Genres with the lowest average word counts:
    - Blues: 120 words
    - Jazz: 124 words
    - Indie-alt: 141 words
- The song with the highest word count is in the hiphop category:
    - Song name: Rap God 
    - Artist: Eminem
    - Word count: 909 words
- The song with the lowest word count is in the rock category:
    - Song name: Golf Hill Drive
    - Artist: Boys Life
    - Word count: 26 words

# Examine unique word counts by genre

In [30]:
# Describe the distribution of unique word counts for each cateogry
unique_word_distributions = {}
blues_unique_words = nlp_df[nlp_df['category']=='blues']['unique_word_count'].describe()
unique_word_distributions['blues'] = blues_unique_words

classical_unique_words = nlp_df[nlp_df['category']=='classical']['unique_word_count'].describe()
unique_word_distributions['classical'] = classical_unique_words

country_unique_words = nlp_df[nlp_df['category']=='country']['unique_word_count'].describe()
unique_word_distributions['country'] = country_unique_words

funk_unique_words = nlp_df[nlp_df['category']=='funk']['unique_word_count'].describe()
unique_word_distributions['funk'] = funk_unique_words

hiphop_unique_words = nlp_df[nlp_df['category']=='hiphop']['unique_word_count'].describe()
unique_word_distributions['hiphop'] = hiphop_unique_words

indie_alt_unique_words = nlp_df[nlp_df['category']=='indie_alt']['unique_word_count'].describe()
unique_word_distributions['indie_alt'] = indie_alt_unique_words

jazz_unique_words = nlp_df[nlp_df['category']=='jazz']['unique_word_count'].describe()
unique_word_distributions['jazz'] = jazz_unique_words

metal_unique_words = nlp_df[nlp_df['category']=='metal']['unique_word_count'].describe()
unique_word_distributions['metal'] = metal_unique_words

pop_unique_words = nlp_df[nlp_df['category']=='pop']['unique_word_count'].describe()
unique_word_distributions['pop'] = pop_unique_words

punk_unique_words = nlp_df[nlp_df['category']=='punk']['unique_word_count'].describe()
unique_word_distributions['punk'] = punk_unique_words

rnb_unique_words = nlp_df[nlp_df['category']=='rnb']['unique_word_count'].describe()
unique_word_distributions['rnb'] = rnb_unique_words

rock_unique_words = nlp_df[nlp_df['category']=='rock']['unique_word_count'].describe()
unique_word_distributions['rock'] = rock_unique_words

romance_unique_words = nlp_df[nlp_df['category']=='romance']['unique_word_count'].describe()
unique_word_distributions['romance'] = romance_unique_words

soul_unique_words = nlp_df[nlp_df['category']=='soul']['unique_word_count'].describe()
unique_word_distributions['soul'] = soul_unique_words
len(unique_word_distributions)

14

In [31]:
# Print the distributions for each genre
unique_word_distributions

{'blues': count    424.000000
 mean      53.707547
 std       19.685094
 min       25.000000
 25%       39.000000
 50%       51.000000
 75%       64.000000
 max      197.000000
 Name: unique_word_count, dtype: float64,
 'classical': count     93.000000
 mean      78.236559
 std       64.720872
 min       25.000000
 25%       44.000000
 50%       61.000000
 75%       90.000000
 max      503.000000
 Name: unique_word_count, dtype: float64,
 'country': count    1257.000000
 mean       74.820207
 std        24.007187
 min        25.000000
 25%        60.000000
 50%        73.000000
 75%        87.000000
 max       264.000000
 Name: unique_word_count, dtype: float64,
 'funk': count    282.000000
 mean      73.092199
 std       38.428545
 min       26.000000
 25%       50.000000
 50%       63.000000
 75%       83.750000
 max      257.000000
 Name: unique_word_count, dtype: float64,
 'hiphop': count    680.000000
 mean     173.770588
 std       74.667690
 min       28.000000
 25%      123.000

In [32]:
# Create a dictionary of mean unique word distributions
categories = list(unique_word_distributions.keys())
means = []
for category in unique_word_distributions:
    means.append(unique_word_distributions[category]['mean'])
mean_unique_word_counts = dict(zip(categories, means))
mean_unique_word_counts = dict(sorted(mean_unique_word_counts.items(), key=lambda item: item[1], reverse=True))
mean_unique_word_counts

{'hiphop': 173.77058823529413,
 'rnb': 85.18508287292818,
 'classical': 78.23655913978494,
 'country': 74.82020684168656,
 'funk': 73.09219858156028,
 'soul': 70.64476885644768,
 'punk': 70.07029876977153,
 'pop': 69.74891774891775,
 'metal': 69.17594654788418,
 'rock': 63.481632653061226,
 'jazz': 60.656565656565654,
 'romance': 59.55913978494624,
 'indie_alt': 57.984848484848484,
 'blues': 53.70754716981132}

In [36]:
# Find songs with the highest and lowest unique word counts
print('Song with the most unique words:')
display(nlp_df[nlp_df['unique_word_count']==max(nlp_df['unique_word_count'])][['song', 'artist', 'category', 'lyrics', 'unique_word_count']])

print('Genres with songs that have the minimum unique word count of 25 words:')
minimum_genres = list(nlp_df[nlp_df['unique_word_count']==min(nlp_df['unique_word_count'])]['category'])
print(list(set(minimum_genres)), '\n')

min_word_count_df = nlp_df[nlp_df['unique_word_count']==min(nlp_df['unique_word_count'])]
print(f'{len(min_word_count_df)} songs have the minimum unique word count of 25 words.')

for genre in list(set(minimum_genres)):
    print(genre, minimum_genres.count(genre))

Song with the most unique words:


song artist category  \
1262  Momentum (feat. Black Thought & Benny The Butc...   Russ   hiphop   

                                                 lyrics  unique_word_count  
1262   Yeah yeah   Dont compare me to employees who ...                544

Genres with songs that have the minimum unique word count of 25 words:
['rock', 'country', 'punk', 'romance', 'classical', 'indie_alt', 'blues', 'metal'] 

25 songs have the minimum unique word count of 25 words.
rock 5
country 3
punk 1
romance 2
classical 1
indie_alt 8
blues 2
metal 3


In [34]:
# Create DataFrame of unique word count distributions
ranges = []
for category in unique_word_distributions:
    ranges.append(unique_word_distributions[category]['max'] - unique_word_distributions[category]['min'])

columns = ['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max', 'range']
df_rows = []
for index, category in enumerate(unique_word_distributions):
    distribution = list(unique_word_distributions[category])
    distribution.extend([ranges[index]])
    df_rows.append(distribution)
unique_word_count_df = pd.DataFrame(df_rows, index=list(unique_word_distributions.keys()), columns=columns)
unique_word_count_df

count        mean        std   min    25%    50%     75%    max  \
blues       424.0   53.707547  19.685094  25.0   39.0   51.0   64.00  197.0   
classical    93.0   78.236559  64.720872  25.0   44.0   61.0   90.00  503.0   
country    1257.0   74.820207  24.007187  25.0   60.0   73.0   87.00  264.0   
funk        282.0   73.092199  38.428545  26.0   50.0   63.0   83.75  257.0   
hiphop      680.0  173.770588  74.667690  28.0  123.0  166.0  216.00  544.0   
indie_alt   660.0   57.984848  24.801422  25.0   41.0   54.0   68.00  253.0   
jazz        198.0   60.656566  42.117834  26.0   39.0   51.0   65.00  349.0   
metal       898.0   69.175947  26.775840  25.0   51.0   64.0   82.00  220.0   
pop         693.0   69.748918  30.938040  26.0   53.0   64.0   78.00  360.0   
punk        569.0   70.070299  30.317486  25.0   51.0   66.0   82.00  315.0   
rnb         362.0   85.185083  33.778347  31.0   61.0   79.0  100.00  220.0   
rock       1225.0   63.481633  25.927782  25.0   48.0   59.0   74.00  308.0   
romance     279.0   59.559140  21.160127  25.0   45.5   57.0   71.00  166.0   
soul        411.0   70.644769  30.872600  26.0   50.0   64.0   82.00  238.0   

           range  
blues      172.0  
classical  478.0  
country    239.0  
funk       231.0  
hiphop     516.0  
indie_alt  228.0  
jazz       323.0  
metal      195.0  
pop        334.0  
punk       290.0  
rnb        189.0  
rock       283.0  
romance    141.0  
soul       212.0

In [35]:
# Save DataFrame to CSV
unique_word_count_df.to_csv('../Data/Analysis/unique_word_count.csv')

# Conclusions about unique word distributions by genre
- The hiphop genre has the highest mean unique word count of 174 words
- The blues genre has the lowest mean unique word count of 54 words
- Genres with the highest average unique word counts:
    - Hiphop: 174 unique words
    - R&B: 85 unique words
    - Classical: 78 unique words
- Genres with the lowest average unique word counts:
    - Blues: 54 unique words
    - Indie-alt: 58 unique words
    - Jazz: 61 unique words
- The song with the highest unique word count is in the romance category:
    - Song name: Momentum 
    - Artist: Russ
    - Unique word count: 544 words
- The indie-alt genre had the most songs (8) with the minimum unique word count of 25 words
    - The rock genre had the second most songs (5) with the minimum unique word count
- The hiphop, R&B, funk, soul, pop, and jazz genres had the least songs (0) with the minimum unique word count of 25 words
    - The classical and punk genres had the second least songs (1) with the minimum unique word count